<a href="https://colab.research.google.com/github/Sim-mi-gyeong/DeepLearning/blob/main/pytorch_%EB%AA%A8%EB%8D%B8_%EB%A7%A4%EA%B0%9C%EB%B3%80%EC%88%98_%EC%B5%9C%EC%A0%81%ED%99%94(Optimization).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 모델 학습 과정: 각 반복 단계(epoch)에서 모델은 출력을 추측 
# -> 추측과 정답 사이의 오류(손실(loss)) 계산
# -> 매개변수에 대한 오류의 도함수(derivative) 수집
# -> 경사하강법을 사용해 파라미터 최적화

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [ ]:
training_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.ReLU()
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = selfe.linear_relu_stack(x)
    return logits

model = NeuralNetwork()

In [ ]:
# 하이퍼파라미터(Hyperparameter): 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수
# 에포크(epoch) 수: 데이터셋을 반복하는 횟수
# 배치크기(batch size): 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플 수
# 학습율(learning rate): 각 배치/에포크에서 모델의 매개변수를 조절하는 비율
#                     -> 값이 작을수록 학습 속도 느려짐, 값이 크면 학습 중 예측할 수 없는 동작 발생 가능

learning_rate = 1e-3
batch_size = 64
epochs = 5

In [ ]:
# 최적화 단계(Optimization Loop)

# 하이퍼파라미터 설정한 뒤 최적화 단계를 통해 모델 학습/최적화 가능
# 최적화 단계의 각 반복(iteration)=에폭

# 하나의 에폭
# 학습 단계(train loop): 학습용 데이터셋을 반복(iteration)하고 최적의 매개변수로 수렴
# 검증/테스트 단계(validation/test loop): 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋 반복

In [ ]:
# 손실함수(loss function)

# 학습용 data 제공 -> 학습되지 않은 신경망은 정답 제공 X
# 손실 함수: 획득 결과와 실제 값 사이의 틀린 정도 측정 -> 학습 중 이 값을 최소화하고자 함.
# 주어진 data sample을 입력으로 계산한 예측과 정답(label)을 비교 -> 손실(loss) 계산

# 회귀 문제 -> nn.MSELoss(평균제곱오차)
# 분류 -> nn.NLLLoss(음의 로그 우도(Negative Log Likelihood)), nn.LogSoftmax
#      -> nn.CrossEntropyLoss: nn.LogSoftmax + nn.NLLLoss

# model의 출력 logit을 nn.CrossEntropyLoss에 전달 -> logit 정규화, 예측 오류 계산
# 손실 함수 초기화
loss_fn = nn.CrossEntropyLoss()

In [ ]:
model.parameters()

<generator object Module.parameters at 0x7f79f116ab50>

In [ ]:
# 옵티마이저(Optimizer)

# 최적화: 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정
# 최적화 알고리즘: SGD(확률적 경사하강법), ADAM, RMSProp
# 최적화 절차는 optimizer 객체에 캡슐화 됨

# 학습 모델의 매개변수와 학습률 파라미터 등록 -> 옵티마이저 초기화
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# 학습 단계(loop)에서 최적화의 세 단계

# optimizer.zero_grad()
# loss.backward()
# optimizer.step()